In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
loans_2007 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Side projects/loans_2007.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
loans_2007.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [5]:
loans_2007.shape

(42538, 52)

The dataset contains 52 columns. Before moving onto the next stage, we need to sort out the features - namely to remove the unncecessary features.

In particular, we need to remove the following kinds of features:


1.   Redundant/repetitive data
2.   Data leakaging featuring (those not available at the start of decision making process)
1.   Not having any effect on payer's ability to pay back the loan (such as user ID)
2.   Poorly formatted & requiring too much of effort to clean up the data





In [0]:
columns_drop = ['id','member_id',
                'funded_amnt','funded_amnt_inv',
                'grade','sub_grade',
                'emp_title','issue_d',
                'zip_code','out_prncp',
                'out_prncp_inv','total_pymnt',
                'total_pymnt_inv','total_rec_prncp',
                'total_rec_int','total_rec_late_fee',
                'recoveries','collection_recovery_fee',
                'last_pymnt_d','last_pymnt_amnt']
loans_2007.drop(columns=columns_drop, axis=0, inplace=True)

In [7]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,other,Personal,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [8]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

Basically, we are interested in only the first two categories of the loan payment - we need to know whether the lender will be able to pay off the loan or not. Therefore, we will be building a classifier that predicts whether or not the certain client is able to pay the loan, and as a result, whether to give them a loan or not.

In [0]:
filter_col = (loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')
loans_2007 = loans_2007[filter_col]

In [0]:
mapping_dict = {
    'loan_status': 
                {'Fully Paid': 1,
                 'Charged Off': 0}
                }
loans_2007 = loans_2007.replace(mapping_dict)

In [11]:
loans_2007.head(3)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


Now let's remove columns that have only one unique value, if there are any. Such columns do not contribute to the prediction since they're redundant.

In [0]:
column_drops = []
original_columns = loans_2007.columns
for col in original_columns:
  column_count = len(loans_2007[col].dropna().unique())
  if column_count == 1:
    column_drops.append(col)
loans_2007.drop(column_drops, axis=1, inplace=True)


In [13]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,Jun-2016,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,Sep-2013,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,Jun-2016,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,Apr-2016,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,Jan-2016,0.0


#Feature Engineering

In [0]:
loans = loans_2007

In [15]:
loans.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

We can see that we are missing some values. Certain columns such as 'emp_length' and 'pub_rec_bankruptcies' have a lot of missing data. We'll deal with missing data in this section.

In [16]:
loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False)

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64

Almost 94% of public bankruptcies records are of the same value. Missing values won't have much of an effect on our prediction abilities, so we can drop them safely.

In [17]:
loans.drop('pub_rec_bankruptcies',axis=1,inplace=True)
loans.dropna(axis=0,inplace=True)
loans.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

We can't use object data type in machine learning, so we need to convert it properly. Let's see what kind of data we have in object columns:

In [18]:
object_df = loans.select_dtypes(include=['object'])
object_df.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


Date columns would require a great amount of feature engineering, and most probably their contribution will not be that big, Therefore, we can drop these columns safely. Let's also take a look at what we have in other columns.

In [19]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for col in cols:
    print(loans[col].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY      76
AK      76
SD      60
VT  

Let's remove the unnecessary columns and start converting some values to numerical values so that we can use them in our Machine Learning model.

In [0]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans.drop(['last_credit_pull_d','addr_state','title','earliest_cr_line'],axis=1, inplace=True)
loans['int_rate'] = loans.int_rate.str.rstrip('%').astype('float')
loans.revol_util = loans.revol_util.str.rstrip('%').astype('float')
loans.replace(mapping_dict,inplace=True)

In [0]:
dummy_df = pd.get_dummies(loans[["home_ownership", "verification_status", "purpose", "term"]])
loans = pd.concat([loans, dummy_df],axis=1)
loans = loans.drop(["home_ownership", "verification_status", "purpose", "term"],axis=1)

In [22]:
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [23]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              37675 non-null float64
int_rate                               37675 non-null float64
installment                            37675 non-null float64
emp_length                             37675 non-null object
annual_inc                             37675 non-null float64
loan_status                            37675 non-null int64
dti                                    37675 non-null float64
delinq_2yrs                            37675 non-null float64
inq_last_6mths                         37675 non-null float64
open_acc                               37675 non-null float64
pub_rec                                37675 non-null float64
revol_bal                              37675 non-null float64
revol_util                             37675 non-null float64
total_acc                              37675 non-null float64
home_ownership_MORTGAGE   

In [24]:
features = loans.drop('loan_status',axis=1)
features.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


Now we are done with preprocessing and we can move on tobuilding a Machine Learning model.

# Machine Learning

Let's split the dataset into training and test sets:

In [0]:
target = loans['loan_status']

In [0]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.2)

Given that have a decent amount of features, and given that we have to classify whether or not to give the loan, we can say that this is a classification issue. There are several options for choosing the classification model, but I have decided to use the Random Forest classifier.

Since we are not sure which hyperparameters would be the best for our dataset, we will use GridSearchCV instance to find the optimal set of hyperparameters.

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
param_grid = {
    'max_depth': [80, 100],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 400, 500, 600, 700]
}
grid = GridSearchCV(estimator = rf, param_grid=param_grid, cv = 3,return_train_score=False,verbose=2)
grid.fit(train_X, train_y)
print(grid.best_params_)
print(grid.best_score_)

Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV] max_depth=80, min_samples_split=8, n_estimators=100 .............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=80, min_samples_split=8, n_estimators=100, total=   3.7s
[CV] max_depth=80, min_samples_split=8, n_estimators=100 .............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s


[CV]  max_depth=80, min_samples_split=8, n_estimators=100, total=   3.9s
[CV] max_depth=80, min_samples_split=8, n_estimators=100 .............
[CV]  max_depth=80, min_samples_split=8, n_estimators=100, total=   3.8s
[CV] max_depth=80, min_samples_split=8, n_estimators=200 .............
[CV]  max_depth=80, min_samples_split=8, n_estimators=200, total=   7.5s
[CV] max_depth=80, min_samples_split=8, n_estimators=200 .............
[CV]  max_depth=80, min_samples_split=8, n_estimators=200, total=   7.4s
[CV] max_depth=80, min_samples_split=8, n_estimators=200 .............
[CV]  max_depth=80, min_samples_split=8, n_estimators=200, total=   7.4s
[CV] max_depth=80, min_samples_split=8, n_estimators=300 .............
[CV]  max_depth=80, min_samples_split=8, n_estimators=300, total=  11.1s
[CV] max_depth=80, min_samples_split=8, n_estimators=300 .............
[CV]  max_depth=80, min_samples_split=8, n_estimators=300, total=  11.2s
[CV] max_depth=80, min_samples_split=8, n_estimators=300 ......

[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed: 29.6min finished


{'max_depth': 80, 'min_samples_split': 12, 'n_estimators': 200}
0.8578301400763283


As we can see, the best hyperparameter set for our Random Forest classifier is the one with the maximum depth of 80, minimum of 12 samples split, and 200 trees used.

Let's see how it works on our test set.

In [0]:
best_rf = grid.best_estimator_
predictions = best_rf.predict(test_X)

In [32]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predictions)
print(accuracy)

0.8552090245520902


The final accuracy is 85,52%, which a bit lower than our training set. Nevertheless, under this model, we are able to correctly identify payback abilities of 85% of clients applying to get the loan. Let's explore the results a bit more to better understand what does 85% accuracy mean for us.

First of all, we are more interested in the rate of people who are able to pay back the loan. Therefore, we want to increase the amount of true positives as much as possible.

In [0]:
results = test_y
results = results.T
results.columns = ['loan_status','predicted']
results.predicted = predictions.T

In [53]:
results.head()

,loan_status,predicted
17663,1.0,1
18363,1.0,1
25692,1.0,1
20444,0.0,1
33100,1.0,1


In [0]:
results['true_positive'] = (results['loan_status']==1) & (results['predicted'] == 1)

In [0]:
results['false_negative'] = (results['loan_status']== 0) & (results['predicted'] == 1)

In [60]:
results.true_positive.value_counts()

True     6434
False    1101
Name: true_positive, dtype: int64

In [62]:
results.false_negative.value_counts()

False    6450
True     1085
Name: false_negative, dtype: int64

As we can see, there are more than 1000 people who were given loans despite their ability to pay money back! While we were able to give loans that would be paid back to more than 6000 people, we would still lose a lot of money from more than 1000 people who didn't pay the money back.

Therefore, we should re-evaluate our ML model so that it will be penalized for wrong assignment of loans to people who can't pay it back. We can achieve that by adjusting class weights for our model, so that it will avoid assigning loan approvals without careful consideration.

In [0]:
new_rf = RandomForestClassifier(n_estimators=100, max_depth=80, min_samples_split=12, class_weight={0:100, 1:1})
new_rf.fit(train_X, train_y)
adjusted_predictions = new_rf.predict(test_X)

In [136]:
accuracy_score(test_y, adjusted_predictions)

0.8177836761778368

In [0]:
results_new = pd.DataFrame(test_y.T)

In [138]:
results_new['predicted'] = adjusted_predictions
results_new['false_negative'] = (results_new['loan_status']== 0) & (results_new['predicted'] == 1)
results_new.head()

,loan_status,predicted,false_negative
17663,1,1,False
18363,1,1,False
25692,1,1,False
20444,0,1,True
33100,1,1,False


In [139]:
results_new['false_negative'].value_counts()

False    6655
True      880
Name: false_negative, dtype: int64


In this case, our goal is to avoid false negatives while still giving out loans to safe customers. We have managed to reduce potential losses by adjusting the weights in our Random Forest classifier.

Even though we have lowered our accuracy, we have managed to decrease the amount of false negatives by 200 people, thus saving a lot of money for our bank! This clearly shows us that sometimes accuracy is not the best metric we should aim for. 
